In [ ]:
!pip install transformers tensorflow datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

# Load the Hugging Face dataset
dataset = load_dataset("carblacac/twitter-sentiment-analysis")

# Split the dataset into train, validation, and test sets
train_data, val_data, test_data = dataset["train"], dataset["validation"], dataset["test"]


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/149985 [00:00<?, ? examples/s]

...

Creating json from Arrow format:   0%|          | 0/30 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/62 [00:00<?, ?ba/s]

Generating train split:   0%|          | 0/119988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/29997 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/61998 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoTokenizer
import re
tokenizer = AutoTokenizer.from_pretrained("carblacac/twitter-sentiment-analysis")
def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()

    # Remove unnecessary characters, punctuation, and special symbols
    text = re.sub(r'[^\w\s]', '', text)

    # Tokenize the text into sequences of words or subwords
    encoded_text = tokenizer(text, padding='max_length', truncation=True, max_length=100)

    return encoded_text['input_ids']

# Preprocess the train, validation, and test data
train_sequences = [preprocess_text(text) for text in train_data["text"]]
val_sequences = [preprocess_text(text) for text in val_data["text"]]
test_sequences = [preprocess_text(text) for text in test_data["text"]]

train_labels = train_data["feeling"]
val_labels = val_data["feeling"]
test_labels = test_data["feeling"]


tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
num_classes = len(set(train_labels))

In [ ]:
import numpy as np
train_sequences=np.array(train_sequences)
val_sequences=np.array(val_sequences)
test_sequences=np.array(test_sequences)

train_labels=np.array(train_labels)
val_labels=np.array(val_labels)
test_labels=np.array(test_labels)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

train_labels = tf.keras.utils.to_categorical(train_labels,num_classes=num_classes)
val_labels = tf.keras.utils.to_categorical(val_labels,num_classes=num_classes)
test_labels = tf.keras.utils.to_categorical(test_labels,num_classes=num_classes)


In [ ]:
vocab_size = tokenizer.vocab_size
embedding_dim=50
maxlen=100

model = Sequential([
    Embedding(vocab_size, embedding_dim, input_length=maxlen),
    Bidirectional(LSTM(units=64, return_sequences=True)),
    Bidirectional(LSTM(units=32)),
    Dense(units=num_classes, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(train_sequences, train_labels, epochs=5, validation_data=(val_sequences, val_labels))


Epoch 1/5
3750/3750 [==============================] - 236s 59ms/step - loss: 0.4843 - accuracy: 0.7677 - val_loss: 0.4436 - val_accuracy: 0.7931
Epoch 2/5
3750/3750 [==============================] - 167s 45ms/step - loss: 0.3973 - accuracy: 0.8181 - val_loss: 0.4477 - val_accuracy: 0.7941
Epoch 3/5
3750/3750 [==============================] - 171s 46ms/step - loss: 0.3426 - accuracy: 0.8474 - val_loss: 0.4637 - val_accuracy: 0.7871
Epoch 4/5
3750/3750 [==============================] - 170s 45ms/step - loss: 0.2883 - accuracy: 0.8748 - val_loss: 0.4854 - val_accuracy: 0.7837
Epoch 5/5
3750/3750 [==============================] - 189s 50ms/step - loss: 0.2375 - accuracy: 0.8990 - val_loss: 0.5870 - val_accuracy: 0.7759


In [ ]:
test_loss, test_acc = model.evaluate(test_sequences, test_labels)
print('Test Accuracy:', test_acc)

1938/1938 [==============================] - 39s 20ms/step - loss: 0.5983 - accuracy: 0.7756
Test Accuracy: 0.7756056785583496
